In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    
from torchvision import models, transforms
import logging
import requests
from io import BytesIO
import os
import cv2

import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.utils
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import shutil
import os
from torch.utils.tensorboard import SummaryWriter
import datetime
from datetime import datetime
import tensorflow as tf

In [ ]:
# Plotting data
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

Bedoeling hieronder is dat ie uit de dataset random een pair pakt (img0 en img1) en eentje pakt die niet tot dezelfde class behoort


In [ ]:
#dataloader for getting triplets from dataset
class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        
    def __getitem__(self,index):
        #img0_tuple = random.choice(self.imageFolderDataset.imgs)
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #search untille requirements are met (same class)
        while True:
          img1_tuple = random.choice(self.imageFolderDataset.imgs)
          if img0_tuple[1] == img1_tuple[1]:
            break
        while True:
          #search untill they arent the same class
          img2_tuple = random.choice(self.imageFolderDataset.imgs)
          if img0_tuple[1] != img2_tuple[1]:
            break
        while True:
          img3_tuple = random.choice(self.imageFolderDataset.imgs)
          if (img3_tuple[1] != img1_tuple[1]) and (img3_tuple[1] != img2_tuple[1]):
            break
        while True:
          img4_tuple = random.choice(self.imageFolderDataset.imgs)
          if (img4_tuple[1] != img1_tuple[1]) and (img4_tuple[1] != img2_tuple[1]) and (img4_tuple[1] != img3_tuple[1]):
            break
        
        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img2 = Image.open(img2_tuple[0])
        img3 = Image.open(img3_tuple[0])
        img4 = Image.open(img4_tuple[0])

      
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)
            img4 = self.transform(img4)


        return img0, img1, img2, img3, img4
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Resize the images and transform to tensors
transformation = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=[0.486, 0.459, 0.408],
                                 std=[0.229, 0.224, 0.225]),transforms.RandomHorizontalFlip(),
                                    
                                    ])

In [ ]:
folder_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Person_reID_baseline_pytorch/Market-1501-v15.09.15/pytorch/train_all")
folder_validationset = datasets.ImageFolder(root="//content/drive/MyDrive/Person_reID_baseline_pytorch/Market-1501-v15.09.15/pytorch/valid")


In [ ]:
train = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transformation)

valid = SiameseNetworkDataset(imageFolderDataset=folder_validationset,
                                        transform=transformation)


In [ ]:
trainloader = DataLoader(train, batch_size=32, num_workers=0, shuffle=True)
validloader = DataLoader(valid, batch_size=32, num_workers=0, shuffle=False)

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
class quintuplet(torch.nn.Module):

    def __init__(self):
        super(quintuplet, self).__init__()
        self.net = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights)


    def forward(self, input1, input2, input3, input4, input5):

        output1 = self.net(input1)
        output2 = self.net(input2)
        output3 = self.net(input3)
        output4 = self.net(input4)
        output5 = self.net(input5)

        return output1, output2, output3, output4, output5

In [ ]:
# define the tripletLoss function
class Quintupletloss(torch.nn.Module):
    """
    Quintuplet loss function.
    Takes 5 data input: one anchor, one positive and two negative examples. The negative examples needs not to be matching the anchor, the positive and each other.
    """
    def __init__(self, margin1=1, margin2=0.25, margin3=0.25):
        super(Quintupletloss, self).__init__()
        self.margin1 = margin1
        self.margin2 = margin2
        self.margin3 = margin3

    def forward(self, anchor, positive, negative1, negative2, negative3):

        squarred_distance_pos = (anchor - positive).pow(2).sum(1)
        squarred_distance_neg = (anchor - negative1).pow(2).sum(1)
        squarred_distance_neg_b = (negative1 - negative2).pow(2).sum(1)
        squarred_distance_neg_c = (negative2 - negative3).pow(2).sum(1)


        quintuplet_loss = \
            F.relu(self.margin1 + squarred_distance_pos - squarred_distance_neg) \
            + F.relu(self.margin2 + squarred_distance_pos - squarred_distance_neg_b) \
            + F.relu(self.margin3 + squarred_distance_pos - squarred_distance_neg_c) 




        return quintuplet_loss.mean()

In [ ]:
cd /content/drive/MyDrive/weets

/content/drive/MyDrive/weets


start training only with the loaded wieghts


In [ ]:
net = quintuplet().cuda()
optimizer = optim.Adam(net.parameters(), lr = 0.0004)
criterion = Quintupletloss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',0.5)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
countert = []
counterv = []
loss_historytrain = [] 
loss_historyvalid = [] 
iteration_numbert= 0
iteration_numbertt= 0

loss_train_items =[]

epochh = 0
epochhh = []

min_valid_loss = 0.05

# Iterate throught the epochs
for epoch in range(60):
    net.train()
    train_loss = 0.0
    epochh +=1
    epochhh.append(epoch)
    # Iterate over batches
    epochh += 1
    for i, (img0, img1, img2, img3, img4) in enumerate(trainloader,0):

        # Send the images to CUDA
        img0, img1, img2, img3, img4 = img0.cuda(), img1.cuda(), img2.cuda(), img3.cuda(), img4.cuda()

        # Zero the gradients
        optimizer.zero_grad(set_to_none=True)

      
        # Pass in the three images into the network 
        output1, output2, output3, output4, output5= net(img0, img1, img2, img3, img4)

        # Pass the outputs into the loss function
        loss_siamese = criterion(output1, output2, output3, output4, output5)

        # Calculate the backpropagation
        loss_siamese.backward()

        # Optimize
        optimizer.step()

        # Every 10 batch print out the loss
        
        train_loss += loss_siamese.item()
        
        

        
        loss_train_items.append(loss_siamese.item())
        if i % 10 == 0 and i>0:
          average = sum(loss_train_items[-9:]) / 10
          print('average last 10 iterations = ', sum(loss_train_items[-9:]) / 10)

    
    epoch_loss = (train_loss /len(trainloader))
    loss_historytrain.append(epoch_loss)
    print('average loss epoch =', (train_loss /len(trainloader)))
    torch.cuda.empty_cache()


    valid_loss= 0.0
    # Iterate over batches
    with torch.no_grad():
      for i, (img0, img1, img2, img3, img4) in enumerate(validloader,0):

          # Send the images to CUDA
          img0, img1, img2, img3, img4 = img0.cuda(), img1.cuda(), img2.cuda(), img3.cuda(), img4.cuda()

        
          # Pass in the three images into the network 
          output1, output2, output3, output4, output5= net(img0, img1, img2, img3, img4)

          # Pass the outputs into the loss function
          loss_valid = criterion(output1, output2, output3, output4, output5)

          # Every 10 batch print out the loss
          
          valid_loss += loss_valid.item()
                   

   
            
  
      print('average valid loss epoch =', (valid_loss /len(validloader)))
      average_valid_loss = valid_loss /len(validloader)
      loss_historyvalid.append(average_valid_loss)
      scheduler.step(average_valid_loss)
      if min_valid_loss > average_valid_loss:
          print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{average_valid_loss:.6f}) \t Saving The Model')
          min_valid_loss = average_valid_loss
          # Saving State Dict
          torch.save({
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, 
           'quinvalid33')

     
      
      torch.cuda.empty_cache()

show_plot(epochhh,loss_historytrain)
show_plot(epochhh,loss_historyvalid)

average last 10 iterations =  247.59681091308593
average last 10 iterations =  169.01825103759765


In [ ]:
#save the data as a checkpoint
torch.save({
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, 
           'quinvalid3')